# Add Features

In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np

from psycopg2.extras import execute_values

1. Historic wins against opponent
2. Conference avg. rank

## Example Data

In [2]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# pull regular season and tourney data to build indexes
query = '''
    SELECT *
    FROM prod.features
    WHERE "Season Type" = 'Regular'
    LIMIT 5
'''

df = pd.read_sql_query(query, conn)
conn.close()

In [3]:
df.shape

(5, 37)

In [4]:
df.head()

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,AvgPointsFor,AvgPointsAgainst,AvgNetPointsFor,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor,TourWins,OpponentTourWins,TourWinsDiff
0,2003,26,1103,1324,1,88,84,0,A,Regular,...,75.0000,78.0000,-3.0000,None,95.0000,68.0,27.00000,0.0,0.0,0.0
1,2003,77,1115,1108,0,50,79,0,H,Regular,...,50.0000,78.5714,-28.5714,None,66.5882,76.0,-9.41177,0.0,0.0,0.0
2,2003,20,1105,1435,0,40,68,0,H,Regular,...,NaN,NaN,NaN,None,80.0000,58.0,22.00000,0.0,4.0,-4.0
3,2003,71,1155,1314,0,66,68,0,H,Regular,...,76.6364,64.5455,12.0909,None,70.7143,65.5,5.21429,5.0,46.0,-41.0
4,2003,30,1156,1103,1,68,65,0,A,Regular,...,69.0000,71.8000,-2.8000,None,81.5000,81.0,0.50000,2.0,0.0,2.0


## Historic Wins Against Opponent

In [5]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# Pull regular season **AND** tourney data to build indexes

query = '''
    SELECT "Season", "DayNum", "Team", "Opponent", "Outcome"
    FROM prod.features

'''
#     WHERE "Season Type" = 'Regular'



df = pd.read_sql_query(query, conn)
conn.close()

In [6]:
df.shape

(170600, 5)

In [7]:
df.head()

,Season,DayNum,Team,Opponent,Outcome
0,2003,26,1103,1324,1.0
1,2003,77,1115,1108,0.0
2,2003,20,1105,1435,0.0
3,2003,71,1155,1314,0.0
4,2003,30,1156,1103,1.0


In [8]:
# Test - filtering off team 1102 and opponent 1204, 1140

#df[['Season', 'Team' == 1102, 'Opponent' == 1204, 'Outcome']]
df[(df['Team'] == 1102) & ((df['Opponent'] == 1204) | (df['Opponent'] == 1140))]



# Could do historical record up to that point - for first game just missing game (NULL type in postgress) - if not a value previously, may default to null

#either season less than current season and if season equal daynum less than current game else don't include

,Season,DayNum,Team,Opponent,Outcome
7901,2006,66,1102,1140,1.0
7902,2006,96,1102,1140,0.0
10168,2003,91,1102,1140,0.0
10218,2003,117,1102,1140,0.0
13956,2004,82,1102,1140,1.0
13957,2004,112,1102,1140,0.0
17741,2007,89,1102,1140,0.0
17742,2007,120,1102,1140,0.0
30074,2003,58,1102,1204,1.0
30482,2008,117,1102,1140,0.0


In [9]:
# Wins vs. each opponent

# Could separate seasons, but commenting out
wins = df[['Season', 'DayNum', 'Team', 'Opponent', 'Outcome']].groupby(by=['Season', 'DayNum', 'Team', 'Opponent']).agg(['sum', 'count'])

# # Aggregating separate seasons
# wins = df[['Team', 'Opponent', 'Outcome']].groupby(by=["Team", "Opponent"]).agg(['sum', 'count'])

wins.reset_index(inplace=True)
wins.columns = [' '.join(col).strip() for col in wins.columns.values]
wins.rename(columns={'Outcome sum': 'wins_v_opp', 'Outcome count': 'games_v_opp'}, inplace=True)
wins['winpct_v_opp'] = wins['wins_v_opp'] / wins['games_v_opp']

In [10]:
wins.shape

(170600, 7)

In [11]:
wins.head()

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp
0,2003,10,1104,1328,1.0,1,1.0
1,2003,10,1272,1393,1.0,1,1.0
2,2003,10,1328,1104,0.0,1,0.0
3,2003,10,1393,1272,0.0,1,0.0
4,2003,11,1186,1458,0.0,1,0.0


In [12]:
# Test - filtering off team 1102 and opponent 1204, 1140

wins[(wins['Team'] == 1102) & ((wins['Opponent'] == 1204) | (wins['Opponent'] == 1140))]

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp
1526,2003,34,1102,1204,1.0,1,1.0
3000,2003,58,1102,1204,1.0,1,1.0
5884,2003,91,1102,1140,0.0,1,0.0
8070,2003,117,1102,1140,0.0,1,0.0
14232,2004,82,1102,1140,1.0,1,1.0
17000,2004,112,1102,1140,0.0,1,0.0
23596,2005,82,1102,1140,1.0,1,1.0
26446,2005,112,1102,1140,1.0,1,1.0
31790,2006,66,1102,1140,1.0,1,1.0
34492,2006,96,1102,1140,0.0,1,0.0


In [13]:
wins.dtypes

Season            int64
DayNum            int64
Team              int64
Opponent          int64
wins_v_opp      float64
games_v_opp       int64
winpct_v_opp    float64
dtype: object

In [14]:
# Creating combined season and day_num column

wins['season_daynum'] = wins['Season'].astype(str) + '_' + wins['DayNum'].astype(str)

# wins['season_daynum'] = wins.apply(lambda x:'%s_%s' % (x['Season'], x['DayNum']), axis = 1)
# wins['season_daynum'] = wins['season_daynum'].astype(int)

wins.head()

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp,season_daynum
0,2003,10,1104,1328,1.0,1,1.0,2003_10
1,2003,10,1272,1393,1.0,1,1.0,2003_10
2,2003,10,1328,1104,0.0,1,0.0,2003_10
3,2003,10,1393,1272,0.0,1,0.0,2003_10
4,2003,11,1186,1458,0.0,1,0.0,2003_11


In [15]:
# Attempting to convert "season_daynum" to type string

wins['season_daynum'] = wins['season_daynum'].astype(str)
# wins['season_daynum'] = wins['season_daynum'].astype('|S')

# wins['season_daynum'] = wins['season_daynum'].str.split(',') 

wins.dtypes

Season             int64
DayNum             int64
Team               int64
Opponent           int64
wins_v_opp       float64
games_v_opp        int64
winpct_v_opp     float64
season_daynum     object
dtype: object

In [16]:
wins.head()

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp,season_daynum
0,2003,10,1104,1328,1.0,1,1.0,2003_10
1,2003,10,1272,1393,1.0,1,1.0,2003_10
2,2003,10,1328,1104,0.0,1,0.0,2003_10
3,2003,10,1393,1272,0.0,1,0.0,2003_10
4,2003,11,1186,1458,0.0,1,0.0,2003_11


In [17]:
# Test - filtering off team 1102 and opponent 1204, 1140, season_daynum < 2008_86

wins[(wins['Team'] == 1102) & ((wins['Opponent'] == 1204) | (wins['Opponent'] == 1140)) & (wins['season_daynum'] < '2008_86')]

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp,season_daynum
1526,2003,34,1102,1204,1.0,1,1.0,2003_34
3000,2003,58,1102,1204,1.0,1,1.0,2003_58
5884,2003,91,1102,1140,0.0,1,0.0,2003_91
8070,2003,117,1102,1140,0.0,1,0.0,2003_117
14232,2004,82,1102,1140,1.0,1,1.0,2004_82
17000,2004,112,1102,1140,0.0,1,0.0,2004_112
23596,2005,82,1102,1140,1.0,1,1.0,2005_82
26446,2005,112,1102,1140,1.0,1,1.0,2005_112
31790,2006,66,1102,1140,1.0,1,1.0,2006_66
34492,2006,96,1102,1140,0.0,1,0.0,2006_96


In [19]:
# Making data cumulative up to, but not including current day.  Inital matchups will have no data.  Takes a while to run.


# Subset data for testing

# test_slice = wins.head(10000)


def cumulative_wins_v_opp(row):
    return wins[(wins.Team == row.Team) & (wins.Opponent == row.Opponent) & 
                      (wins.season_daynum < row.season_daynum)].wins_v_opp.sum()

def cumulative_games_v_opp(row):
    return wins[(wins.Team == row.Team) & (wins.Opponent == row.Opponent) & 
                      (wins.season_daynum < row.season_daynum)].games_v_opp.sum()

# wins['cum_games_v_opp'] = wins.apply(cumulative_wins_v_opp, axis=1)

# for index, row in wins.iterrows():
# #     print(index)

#     wins['cum_wins_v_opp'] = wins[(wins.Team == row.Team) & (wins.season_daynum < row.season_daynum)].wins_v_opp.sum()
#     wins['cum_games_v_opp'] = wins[(wins.Team == row.Team) & (wins.season_daynum < row.season_daynum)].games_v_opp.sum()

# test_slice['cum_wins_v_opp'] = test_slice.apply(cumulative_wins_v_opp, axis=1)
# test_slice['cum_games_v_opp'] = test_slice.apply(cumulative_games_v_opp, axis=1)

wins['cum_wins_v_opp'] = wins.apply(cumulative_wins_v_opp, axis=1)
wins['cum_games_v_opp'] = wins.apply(cumulative_games_v_opp, axis=1)

# test_slice.shape
# test_slice
wins

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp,season_daynum,cum_wins_v_opp,cum_games_v_opp
0,2003,10,1104,1328,1.0,1,1.0,2003_10,0.0,0
1,2003,10,1272,1393,1.0,1,1.0,2003_10,0.0,0
2,2003,10,1328,1104,0.0,1,0.0,2003_10,0.0,0
3,2003,10,1393,1272,0.0,1,0.0,2003_10,0.0,0
4,2003,11,1186,1458,0.0,1,0.0,2003_11,0.0,0
5,2003,11,1208,1400,0.0,1,0.0,2003_11,0.0,0
6,2003,11,1266,1437,1.0,1,1.0,2003_11,0.0,0
7,2003,11,1296,1457,1.0,1,1.0,2003_11,0.0,0
8,2003,11,1400,1208,1.0,1,1.0,2003_11,0.0,0
9,2003,11,1437,1266,0.0,1,0.0,2003_11,0.0,0


In [40]:
# Testing 2018 data

wins[(wins['Season'] == 2018)]

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp,season_daynum,cum_wins_v_opp,cum_games_v_opp
155234,2018,11,1104,1272,1.0,1,1.0,2018_11,0.0,1
155235,2018,11,1106,1280,0.0,1,0.0,2018_11,0.0,3
155236,2018,11,1107,1233,1.0,1,1.0,2018_11,0.0,2
155237,2018,11,1108,1261,0.0,1,0.0,2018_11,0.0,1
155238,2018,11,1110,1243,0.0,1,0.0,2018_11,0.0,0
155239,2018,11,1112,1319,1.0,1,1.0,2018_11,12.0,12
155240,2018,11,1113,1226,1.0,1,1.0,2018_11,2.0,2
155241,2018,11,1115,1218,0.0,1,0.0,2018_11,0.0,5
155242,2018,11,1116,1359,1.0,1,1.0,2018_11,0.0,0
155243,2018,11,1117,1191,0.0,1,0.0,2018_11,0.0,0


In [44]:
# Generating cumulative wins percentages data

wins['cum_winpct_v_opp'] = wins['cum_wins_v_opp'] / wins['cum_games_v_opp']

wins[(wins['Season'] == 2018)].head(50)

,Season,DayNum,Team,Opponent,wins_v_opp,games_v_opp,winpct_v_opp,season_daynum,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
155234,2018,11,1104,1272,1.0,1,1.0,2018_11,0.0,1,0.000000
155235,2018,11,1106,1280,0.0,1,0.0,2018_11,0.0,3,0.000000
155236,2018,11,1107,1233,1.0,1,1.0,2018_11,0.0,2,0.000000
155237,2018,11,1108,1261,0.0,1,0.0,2018_11,0.0,1,0.000000
155238,2018,11,1110,1243,0.0,1,0.0,2018_11,0.0,0,NaN
155239,2018,11,1112,1319,1.0,1,1.0,2018_11,12.0,12,1.000000
155240,2018,11,1113,1226,1.0,1,1.0,2018_11,2.0,2,1.000000
155241,2018,11,1115,1218,0.0,1,0.0,2018_11,0.0,5,0.000000
155242,2018,11,1116,1359,1.0,1,1.0,2018_11,0.0,0,NaN
155243,2018,11,1117,1191,0.0,1,0.0,2018_11,0.0,0,NaN


In [75]:
# Preparing to load data

to_load = wins[['Season', 'DayNum', 'Team', 'Opponent', 'cum_wins_v_opp', 'cum_games_v_opp', 'cum_winpct_v_opp']].copy()


to_load = to_load.where(pd.notnull(to_load), None)


to_load

,Season,DayNum,Team,Opponent,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
0,2003,10,1104,1328,0,0,None
1,2003,10,1272,1393,0,0,None
2,2003,10,1328,1104,0,0,None
3,2003,10,1393,1272,0,0,None
4,2003,11,1186,1458,0,0,None
5,2003,11,1208,1400,0,0,None
6,2003,11,1266,1437,0,0,None
7,2003,11,1296,1457,0,0,None
8,2003,11,1400,1208,0,0,None
9,2003,11,1437,1266,0,0,None


In [77]:
datarows = []
for i in to_load.itertuples(index=False):
    row = (int(i.Season), int(i.DayNum), int(i.Team), int(i.Opponent), int(i.cum_wins_v_opp), 
               int(i.cum_games_v_opp), i.cum_winpct_v_opp)
    datarows.append(row)


In [78]:
alter = '''
    ALTER TABLE prod.features
      ADD COLUMN IF NOT EXISTS "cum_wins_v_opp" REAL,
      ADD COLUMN IF NOT EXISTS "cum_games_v_opp" REAL,
      ADD COLUMN IF NOT EXISTS "cum_winpct_v_opp" REAL
'''

update = '''
    UPDATE prod.features as f
       SET "cum_wins_v_opp" = data."cum_wins_v_opp",
           "cum_games_v_opp" = data."cum_games_v_opp",
           "cum_winpct_v_opp" = data."cum_winpct_v_opp"
      FROM (VALUES %s) AS data (
             "Season",
             "DayNum",
             "Team",
             "Opponent",
             "cum_wins_v_opp",
             "cum_games_v_opp",
             "cum_winpct_v_opp"
             )
     WHERE f."Season" = data."Season"
       and f."DayNum" = data."DayNum"
       and f."Team" = data."Team"
       and f."Opponent" = data."Opponent"

'''
#        and f."Season Type" = 'NCAA Tourney'
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

c = conn.cursor()
c.execute(alter)
# execute_values(c, update, datarows, page_size=8000)
execute_values(c, update, datarows, page_size=200000)
conn.commit()
conn.close()

In [79]:
# Testing uploaded data

conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# pull regular season and tourney data to build indexes
query = '''
    SELECT *
    FROM prod.features
    WHERE "Season Type" = 'Regular'
    AND "Season" = 2018
    LIMIT 500
'''

df = pd.read_sql_query(query, conn)
conn.close()

df.head(500)

,Season,DayNum,Team,Opponent,Outcome,Score,OpponentScore,NumOT,WLoc,Season Type,...,SeedDiff,OpponentAvgPointsFor,OpponentAvgPointsAgainst,OpponentAvgNetPointsFor,TourWins,OpponentTourWins,TourWinsDiff,cum_wins_v_opp,cum_games_v_opp,cum_winpct_v_opp
0,2018,11,1146,1124,0,66,107,0,H,Regular,...,None,NaN,NaN,NaN,0.0,10.0,-10.0,0.0,0.0,NaN
1,2018,11,1407,1315,0,80,83,0,N,Regular,...,None,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
2,2018,11,1132,1180,1,78,69,0,A,Regular,...,None,NaN,NaN,NaN,0.0,1.0,-1.0,0.0,0.0,NaN
3,2018,12,1417,1210,1,63,60,0,N,Regular,...,None,NaN,NaN,NaN,46.0,22.0,24.0,1.0,1.0,1.000000
4,2018,11,1313,1120,0,74,102,0,H,Regular,...,None,NaN,NaN,NaN,1.0,12.0,-11.0,0.0,0.0,NaN
5,2018,11,1324,1236,1,85,71,0,H,Regular,...,None,NaN,NaN,NaN,1.0,0.0,1.0,12.0,17.0,0.705882
6,2018,11,1220,1119,1,77,74,0,H,Regular,...,None,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
7,2018,11,1113,1226,1,94,74,0,H,Regular,...,None,NaN,NaN,NaN,5.0,0.0,5.0,2.0,2.0,1.000000
8,2018,11,1165,1393,0,45,77,0,H,Regular,...,None,NaN,NaN,NaN,2.0,52.0,-50.0,0.0,11.0,0.000000
9,2018,11,1192,1371,0,68,90,0,H,Regular,...,None,NaN,NaN,NaN,0.0,15.0,-15.0,2.0,5.0,0.400000


In [68]:
df.shape

(500, 40)

In [ ]:
# for tourney features, if never matched - say 50/50 matchup

# Winning / Losing Field Goal Attempts

In [ ]:
# Winning / Losing Field Goal Attempts - Percent of field goals attempted by winning vs. losing teams

conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# Pull regular season **AND** tourney data to build indexes

query = '''
    SELECT "Season", "DayNum", "WTeamID", "LTeamID", "WFGA", "LFGA", "WFGM", "WFGA"
    FROM prod."RegSeasonDetailedResults"
'''
#     WHERE "Season Type" = 'Regular'



df = pd.read_sql_query(query, conn)
conn.close()




In [ ]:
df.shape

In [ ]:
df.head()

## Conference Avg. Rank

In [ ]:
conn = pg.connect(database='postgres',
                  user='postgres',
                  password='w207final',
                  host='35.185.225.167')

# Pull regular season **AND** tourney data to build indexes


query = '''
    SELECT "Season", "ConfAbbrev", "DayNum", "WTeamID", "LTeamID"
    FROM prod."ConferenceTourneyGames"

'''
#WHERE "Season Type" = 'Regular'



df = pd.read_sql_query(query, conn)
conn.close()

In [ ]:
# take avg rank from features and group by conference teams are in for a year

# avg of "avg daily rankings"


# join season, team id for conference